<center>
<h1>INF582: INTRODUCTION TO TEXT MINING AND NLP</h1>
<h2>Lab Session 2: Unsupervised Keyword Extraction</h2>
<h4>Lecture: Prof. Michalis Vazirgiannis<br>
Lab: Moussa Kamal Eddine and Hadi Abdine</h4>
<h5>Friday, January 14, 2022</h5>
<h4><b>Student Name:</b> </h4>
<br>
</center>

<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit to Moodle a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>January 21, 2022 11:59 PM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).
</p>
<hr style="border:5px solid gray"> </hr>


<h3><b>1. Learning Objective</b></h2>
<p style="text-align: justify;">
In this lab, you will learn how methods from social network analysis can be applied to word co- occurrence networks to <b>extract keywords</b>. Keyword extraction is a fundamental NLP task used in many areas like information retrieval (search engines), summarization, natural language generation, visualization... Today, we will focus on <b>unsupervised single-document keyword extraction</b>.
<br><br>
<b>Notation:</b> in what follows, G(V, E) is a graph with |V| nodes (or vertices) and |E| edges (or links). N (v, U ) designates the immediate neighbors of v in U ⊂ V.
<br><br>
<b>Igraph:</b> the nodes and edges of a graph g can be accessed collectively or individually (through indexing), along with their attributes, such as ’name’ or ’weight’, via, e.g., g.vs[’name’] or g.es[index][’weight’]. Documentation of all methods and functions can be found at <a href='http://igraph.org/python/doc/igraph.GraphBase-class.html'>http://igraph.org/python/doc/igraph.GraphBase-class.html</a>.
</p>

<h3><b>2. Text Preprocessing</b></h2>
<p style="text-align: justify;">
Before constructing a word co-occurrence network, the document needs to be cleaned. The standard steps include (1) conversion to lower case, (2) punctuation removal, (3) tokenization, and (4) stopword removal. Additionally, for keyword extraction, (5) part-of-speech-based filtering (e.g., retaining only nouns and adjectives) and (6) stemming (“winner”, “winning”, and “win” → “win”) can be useful. These steps are implemented in the clean text simple function, found within the next cell.
</p>

<h3><b>3. Word co-occurrence networks</b></h2>
<p style="text-align: justify;">
There are many ways to represent text as a graph. Today, we will use the classical statistical approach of [<a href='https://www.researchgate.net/publication/200042361_TextRank_Bringing_Order_into_Text'>Mihalcea et Tarau, 2004</a>], based on the distributional hypothesis (“We shall know a word by the company it keeps” [<a href='https://www.tandfonline.com/doi/pdf/10.1080/00437956.1954.11659520'>Harris, 1954</a>]). This method applies a fixed-size sliding window of size W over the text from start to finish (<a href='https://safetyapp.shinyapps.io/GoWvis/'>demo</a> [<a href='https://aclanthology.org/P16-4026/'>Tixier et al., 2016</a>]). Each unique term in the document is represented by a node of the graph, and two nodes are linked by an edge if the terms they represent co-occur within the window. Edge weights are co-occurrence counts. Unlike the vector space model that assumes term independence, this representation captures term dependency, and even term order, if directed edges are used (see the next figure)
<center>
<img width='500px' src='https://am3pap003files.storage.live.com/y4m3vawSJNCOJlgUoeKlGOFqs2jcBkOJr-nZWegwjwGy9_4R0hDcbunK6jMBq3yXar8tRjdGd4tcbZwhpHQi2L6gMsKyx1tYUNinwLp9yqS9LK--NFIreN_V8MEQywu1TKarAabFVlWUQI2Y32z7VLacAzQ2sY5p6v9vVzeWbx6EqQTWzVX8l1YaykniEQFvbat?width=1094&height=870&cropmode=none' /><br>
<b>Figure 1:</b> Word co-occurrence network example. Non-(nouns and adjectives) in italic. Words have been stemmed. W = 4.
</center>
</p>






### Importing libraries and downloading the data:

In [ ]:
!pip install python-igraph

import re 
import itertools
import operator
import copy
import igraph
import heapq
import string
import os
import string
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer

# requires nltk 3.2.1
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag 
nltk.download('maxent_treebank_pos_tagger')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

import warnings
warnings.filterwarnings("ignore")

!wget -c "https://onedrive.live.com/download?cid=AE69638675180117&resid=AE69638675180117%2199432&authkey=AJxykvJytrd5nLg" -O 'data.zip'
!unzip data.zip


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 1: </b><br>
Fill the gaps in the clean text simple and terms to graph functions in the following cell.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [ ]:
def clean_text_simple(text, my_stopwords, punct, remove_stopwords=True, pos_filtering=True, stemming=True):
    text = text.lower()
    text = ''.join(l for l in text if l not in punct) # remove punctuation (preserving intra-word dashes)
    text = re.sub(' +',' ',text) # strip extra white space
    text = text.strip() # strip leading and trailing white space
    ### FILL THE GAP (split based on whitespace, save results as 'tokens') ###
    if pos_filtering == True:
        # POS-tag and retain only nouns and adjectives
        tagged_tokens = pos_tag(tokens)
        tokens_keep = []
        for item in tagged_tokens:
            if (
            item[1] == 'NN' or
            item[1] == 'NNS' or
            item[1] == 'NNP' or
            item[1] == 'NNPS' or
            item[1] == 'JJ' or
            item[1] == 'JJS' or
            item[1] == 'JJR'
            ):
                tokens_keep.append(item[0])
        tokens = tokens_keep
    if remove_stopwords:
        tokens = [token for token in tokens if token not in my_stopwords]
    if stemming:
        stemmer = nltk.stem.PorterStemmer()
        tokens_stemmed = list()
        for token in tokens:
            tokens_stemmed.append(stemmer.stem(token))
        tokens = tokens_stemmed
    
    return(tokens)


def terms_to_graph(terms, window_size):
    '''This function returns a directed, weighted igraph from lists of list of terms (the tokens from the pre-processed text)
    e.g., ['quick','brown','fox']
    Edges are weighted based on term co-occurence within a sliding window of fixed size 'w'
    '''
    
    from_to = {}

    w = min(window_size, len(terms))
    # create initial complete graph (first w terms)
    terms_temp = terms[0:w]
    indexes = list(itertools.combinations(range(w), r=2))

    new_edges = []

    for my_tuple in indexes:
        new_edges.append(tuple([terms_temp[i] for i in my_tuple]))
    for new_edge in new_edges:
        if new_edge in from_to:
            from_to[new_edge] += 1
        else:
            from_to[new_edge] = 1

    # then iterate over the remaining terms
    for i in range(w, len(terms)):
        # term to consider
        considered_term = terms[i]
        # all terms within sliding window
        terms_temp = terms[(i - w + 1):(i + 1)]

        # edges to try
        candidate_edges = []
        for p in range(w - 1):
            candidate_edges.append((terms_temp[p], considered_term))

        for try_edge in candidate_edges:

            # if not self-edge
            if try_edge[1] != try_edge[0]:

                ### FILL THE GAPS (if edge has already been seen, update its weight, else create it and assign it a unit weight) ###

    # create empty graph
    g = igraph.Graph(directed=True)

    # add vertices
    g.add_vertices(sorted(set(terms)))

    # add edges, direction is preserved since the graph is directed
    g.add_edges(list(from_to.keys()))

    # set edge and vertice weights
    g.es['weight'] = list(from_to.values()) # based on co-occurence within sliding window
    g.vs['weight'] = g.strength(weights=list(from_to.values())) # weighted degree

    return (g)


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 2: </b><br>
Use the next cell to build a graph for the text of Figure 1, with a window of size 4. Validate your results by comparing some edges (source, target, and weight) with Figure 1.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [ ]:
stpwds = stopwords.words('english')
punct = string.punctuation.replace('-', '')

my_doc = 'A method for solution of systems of linear algebraic equations \
with m-dimensional lambda matrices. A system of linear algebraic \
equations with m-dimensional lambda matrices is considered. \
The proposed method of searching for the solution of this system \
lies in reducing it to a numerical system of a special kind.'

my_doc = my_doc.replace('\n', '')

# pre-process document
my_tokens = clean_text_simple(my_doc,my_stopwords=stpwds,punct=punct)

### FILL THE GAP (get the graph with a window of size 4)
g = 

# number of edges
print(len(g.es))

# the number of nodes should be equal to the number of unique terms
len(g.vs) == len(set(my_tokens))

edge_weights = []
for edge in g.es:
    source = g.vs[edge.source]['name']
    target = g.vs[edge.target]['name']
    weight = edge['weight']
    edge_weights.append([source, target, weight])

print(edge_weights)



<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 1 (2 points): </b><br>
Evaluate the impact of window size on the density of the graph, defined as $\frac{|E|}{|V|(|V|-1)}$ for a directed graph and accessible via the .density() igraph method. What do you observe?
<hr style="border:10px solid red"> </hr>
</font></h4>
<p style="text-align: justify;">
Note: the density never reaches 1, even when the window includes all terms in the document, because we work with directed graphs. To be complete, a directed graph must have two edges between each node (one in each direction). On the other hand, for undirected graphs, the density is equal to $\frac{2*|E|}{|V|(|V|-1)}$,  and there can be at most one edge between two nodes.</p>

In [ ]:
densities = []
window_sizes = range(2,14)
for w in window_sizes:
    g = terms_to_graph(my_tokens, w)
    ### FILL THE GAP (print density of g and append it to densities) ###

plt.plot(list(window_sizes),densities)  
plt.xlabel('window size')  
plt.ylabel('density')  
plt.title('density VS window size')   
plt.grid(True)    
plt.show()  

<b><h4><font color='green'>
<hr style="border:10px solid red"> </hr>
Answer 1: </b><br>
Your answer here.
<hr style="border:10px solid red"> </hr>
</font></h4>

<h3><b>4. Keyword Extraction</b></h3>
<h4><b>4.1. Influential words</b></h4>
<p style="text-align: justify;">
In social networks, it was shown that <b>influential spreaders</b>, that is, those individuals that can reach the largest part of the network in a given number of steps, are better identified via their core numbers rather than through their PageRank scores, betweenness centralities, or degrees [<a href='https://www.nature.com/articles/nphys1746'>Kitsak et al., 2010</a>]. For instance, a less connected person who is strategically placed in the core of a network will be able to disseminate more than a hub located at the periphery of the network, as shown in Figure 2.<br><p>
<center>
<img width="500px" src='https://am3pap003files.storage.live.com/y4mvS4IHdj8edLEAwBj8mdP9hOUXbCDKA4WLYoi-BnsAkwdVT7Ry2B1l1VU2-MkfKBzTdjrpHHIdTb7PNmyWp0a1WaQdUOyueVIxo7Bd5CV643BCODwjVV-VO2_uiy0n2Ui2X5VStxAQh4jItMseR1kDwIrVHMbxDkyvRjNJYCUY8MawSqSC58ZoASdA_NvHapi?width=644&height=633&cropmode=none'/><br>

<b>Figure 2:</b> Degree vs. PageRank vs. unweighted core number. Node labels indicate degrees. Nodes ∗ and ∗∗ both have same degree (5) and high PageRank scores (resp. in (6.73, 9.05] and (9.05, 11.4]). However, node ∗ lies in a much more central location and is therefore a much better spreader, which is captured by its higher core number (3 vs 1) but not by degree or PageRank (the PageRank score of node ∗∗ is even greater than that of node ∗).
</center>
<br>
<p style="text-align: justify;">
Interestingly, taking into account the cohesiveness information captured by graph degeneracy was shown to vastly improve keyword extraction performance [<a href='http://frncsrss.github.io/papers/rousseau-ecir2015.pdf'>Rousseau et Vazirgiannis 2015</a>, <a href='https://fragkiskos.me/papers/keywords_degeneracy_EMNLP_2016.pdf'>Tixier et al. 2016</a>], meaning that natural language features an important ”social” aspect. Keywords can thus be seen as “influential” words.
</p>
<h4><b>4.2. Graph Degeneracy</b></h4>
<p style="text-align: justify;">
The concept of graph degeneracy was introduced by [<a href='https://www.sciencedirect.com/science/article/pii/037887338390028X'>Seidman, 1983</a>] with the k-core decomposition technique and was first applied to the study of cohesion in social networks.
k-core A core of order k (or k-core) of G is a maximal connected subgraph of G in which every vertex v has at least degree k (i.e., k neighbors).
k-core decomposition As shown in Fig. 3, the k-core decomposition of G is the set of all its cores from 0 (G itself) to kmax (its main core). It forms a hierarchy of nested subgraphs whose cohesiveness and size respectively increase and decrease with k. The core number of a node is the highest order of a k-core subgraph that contains this node. The main core of G is a coarse approximation of its densest subgraph.
<center>
<img width="500px" src='https://am3pap003files.storage.live.com/y4mDZVLxWHI5aHjzQp8GcY49ta1ZZmtlsxxlQPCXLBLcISJJvN82GMzAOIuBGHrHvd7hTSd5ksRnUo9M0srD2qZkLpa4ss7lbylHDugi0gfBjujGV3tGYAI--CH7NOfeo2e077BTEuoyInqZFsyVY87PLt3xQn2PZ1xjEhhnuTKxtckuHAphbtpm6YBV7YEIPmV?width=1000&height=820&cropmode=none'/><br>

<b>Figure 3:</b> Unweighted k-core
</center>
</p>

<p style="text-align: justify;">
Algorithm 1 shows the unweighted k-core algorithm. It involves a pruning process that removes the lowest degree node at each step, where the degree of a node is simply its num- ber of immediate neighbors. By using the sum of the weights of the incident edges as the de- gree, we obtain the weighted k-core algorithm. The unweighted and weighted k-core algorithm can be implemented with very af- fordable time complexities O(|E|) [<a href='https://arxiv.org/abs/cs/0202039'>Batagelj et Zaveršnik, 2002</a>] and O(|E| log |V |) [<a href='https://arxiv.org/abs/cs/0310049'>Batagelj et Zaveršnik, 2003</a>], by using specific strategies and data structures.
<center>
<img width="500px" src='https://am3pap003files.storage.live.com/y4mLnLfA6Zv9S8wiFKe3MOdd1rY8r6AtKJhowzBo8GavSV0lZPHLEyuPuIW9c33JJwWiMCMLdBA4nc6mYbTVdcTmdSWdbfYNefQW-7IoYcW_oFd4qaDoetHI1eAqD2Ec2wi5KYbhW1ZEw18Pi7vc2BLpd3SpZguVaCfg96uCSMwxU0QW0C2VqhCBZnBfVELvbX8?width=758&height=742&cropmode=none'/><br>
</center>
</p>

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 2 (6 points): </b><br>
What is the time complexity of our naive version of the k-core algorithm (Alg. 1), in the unweighted case?
<hr style="border:10px solid red"> </hr>
</font></h4>
<p style="text-align: justify;">
Notes: use big $\mathcal{O}$ notation. Evaluate each line one by one before deriving the overall complexity of the algorithm. You may assume that G is represented by its adjacency list (a list of lists containing the neighbors of each node), available in RAM, and introduce a variable representing the average number of neighbors of a node.</p>






<b><h4><font color='green'>
<hr style="border:10px solid red"> </hr>
Answer 2: </b><br>
Your answer here.
<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 3: </b><br>
Fill the gaps in the core_dec function in next cell to implement Algorithm 1. Use the .strength() and .delete_vertices() igraph methods.
<hr style="border:10px solid blue"> </hr>
</font></h4>
<p style="text-align: justify;">
Note 1: .delete_vertices() automatically removes the edges incident on the node(s) deleted.<br>
Note 2: to get weighted degrees, you need to use the weights argument of .strength().


In [ ]:
def core_dec(g,weighted):
    '''(un)weighted k-core decomposition'''
    # work on clone of g to preserve g 
    gg = copy.deepcopy(g)
    if not weighted:
        gg.vs['weight'] = gg.strength() # overwrite the 'weight' vertex attribute with the unweighted degrees
    # initialize dictionary that will contain the core numbers
    cores_g = dict(zip(gg.vs['name'], [0]*len(gg.vs)))
    
    while len(gg.vs) > 0:
        # find index of lowest degree vertex
        min_degree = min(gg.vs['weight'])
        index_top = gg.vs['weight'].index(min_degree)
        name_top = gg.vs[index_top]['name']
        # get names of its neighbors
        neighbors = gg.vs[gg.neighbors(index_top)]['name']
        # exclude self-edges
        neighbors = [elt for elt in neighbors if elt!=name_top]
        # set core number of lowest degree vertex as its degree
        cores_g[name_top] = min_degree
        ### FILL THE GAP (delete top vertex and its incident edges) ###
        
        if neighbors:
            if weighted: 
                ### FILL THE GAP (compute the new weighted degrees, save results as 'new_degrees')
            else:
                ### FILL THE GAP (same as above but for the basic degree) ###
            # iterate over neighbors of top element
            for neigh in neighbors:
                index_n = gg.vs['name'].index(neigh)
                gg.vs[index_n]['weight'] = max(min_degree,new_degrees[index_n])  
        
    return(cores_g)


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 4: </b><br>
Decompose the graph shown in Fig. 1. For unweighted k-core by filling the following code. Compare your results with the .coreness() igraph method and Fig. 4 below.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [ ]:
# decompose g
core_numbers = core_dec(g,False)
print(core_numbers)

### FILL THE GAP (compare 'core_numbers' with the output of the .coreness() igraph method) ###

# retain main core as keywords
max_c_n = max(core_numbers.values())
keywords = [kwd for kwd, c_n in core_numbers.items() if c_n == max_c_n]
print(keywords)

<h3><b>5. Keyword Extraction</b></h3>
<h4><b>5.1. Data set</b></h4>
<p style="text-align: justify;">
We will use the test set of the Hulth 2003 dataset [<a href='https://www.researchgate.net/publication/2910565_Improved_Automatic_Keyword_Extraction_Given_More_Linguistic_Knowledge'>Hulth, 2003</a>], that you can find inside the data\Hulth2003testing\ folder. This dataset contains 500 scientific paper abstracts. For each abstract in the (abstracts\ folder), human annotators have provided a set of keywords (uncontr\ folder), that we will consider as ground truth. The keywords were freely chosen by the annotators and some of them may not appear in the original text. Thus, getting a perfect score is impossible on this dataset.
</p>
<h4><b>5.2. Baselines</b></h4>
<p style="text-align: justify;">
We will evaluate the performance of the k-core-based approach against that of PageRank (applied on the same graphs) and the vector space representation with TF-IDF coefficients. For each baseline, the top p = 33% percent nodes will be retained as keywords.
</p>

<center>
<img width="500px" src='https://am3pap003files.storage.live.com/y4mYzSlqeB5hdXaulfc1eYrCTb5ZtRH5fcgC91mY5s3rLIpXxJh_WyzB2eEtAFtmpH82uEOTMhKfljgLovpxlPVqCzqgS5ASVDDWK5e28mgp5lgLwH1X25uUyIPfZcKbW0at-adA_cZz2E9D7bcFA5JKAUQYF-yOiZFjl9L-GgHSeGcamEUiuikXKig9jZmuF9w?width=923&height=851&cropmode=none'/><br>

<b>Figure 4:</b> The main core of the graph can be used as the keywords for the document.
</center>
<br>

<p style="text-align: justify;">
<b>Assumption:</b> both for k-core and weighted k-core, we will extract the main core of the graph as key-
words.
</p>

<h4><b>5.3. Performance evaluation</b></h4>
<p style="text-align: justify;">
We will evaluate the performance of the different techniques in terms of macro-averaged precision, recall and F1 score. Precision can be seen as the purity of retrieval while recall measures the completeness of retrieval. Finally, the F-1 score is the harmonic mean of precision and recall. More precisely, these metrics are defined as follows:
<br>
<center>
$precision=\frac{tp}{tp+fp}$,  $recall=\frac{tp}{tp+fn}$, F1-score=$2\frac{precision.recall}{precision+recall}$
</center>

where tp, fp and fn respectively denote the number of true positives (the keywords returned by the system which are also part of the ground truth), false positives (the keywords returned by the system which are not part of the ground truth), and false negatives (the keywords from the ground truth that are not returned by the system). Finally, macro-averaging consists in computing the metrics for each document and calculating the means at the collection level.
</p>


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 5: </b><br>
Put everything together by filling the gaps in the accuracy metrics function and in the following cell and . For the baselines, use the .pagerank() igraph method, and for TF-IDF, the TfidfVectorizer function from the scikit-learn library.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [ ]:
def accuracy_metrics(candidate, truth):
    # true positives ('hits') are both in candidate and in truth
    tp = len(set(candidate).intersection(truth))
    
    # false positives a.k.a. false alarms are in candidate but not in truth
    fp = len([element for element in candidate if element not in truth])
    
    ### FILL THE GAP (compute false negatives a.k.a. misses, save results as 'fn')
    
    ### FILL THE GAPS (compute precision and recall as a function of 'tp', 'fp' and 'fn', save results as 'prec' and 'rec') ###
    
    if prec+rec != 0:
        f1 = 2*prec*rec/(prec+rec)
    else:
        f1 = 0
    
    return (prec, rec, f1)

In [ ]:
stemmer = nltk.stem.PorterStemmer()
stpwds = stopwords.words('english')
punct = string.punctuation.replace('-', '')

##################################
# read and pre-process abstracts #
##################################

path_to_abstracts = './data/Hulth2003testing/abstracts/'
abstract_names = sorted(os.listdir(path_to_abstracts))

abstracts = []
for counter,filename in enumerate(abstract_names):
    # read file
    with open(path_to_abstracts + '/' + filename, 'r') as my_file: 
        text = my_file.read().splitlines()
    text = ' '.join(text)
    # remove formatting
    text = re.sub('\s+', ' ', text)
    abstracts.append(text)
    
    if counter % round(len(abstract_names)/5) == 0:
        print(counter, 'files processed')

abstracts_cleaned = []
for counter,abstract in enumerate(abstracts):
    my_tokens = clean_text_simple(abstract,my_stopwords=stpwds,punct=punct)
    abstracts_cleaned.append(my_tokens)
    
    if counter % round(len(abstracts)/5) == 0:
        print(counter, 'abstracts processed')

###############################################
# read and pre-process gold standard keywords #
###############################################

path_to_keywords = './data/Hulth2003testing/uncontr/'
keywd_names = sorted(os.listdir(path_to_keywords))
   
keywds_gold_standard = []

for counter,filename in enumerate(keywd_names):
    # read file
    with open(path_to_keywords + filename, 'r') as my_file: 
        text = my_file.read().splitlines()
    text = ' '.join(text)
    text =  re.sub('\s+', ' ', text) # remove formatting
    text = text.lower()
    # turn string into list of keywords, preserving intra-word dashes 
    # but breaking n-grams into unigrams
    keywds = text.split(';')
    keywds = [keywd.strip().split(' ') for keywd in keywds]
    keywds = [keywd for sublist in keywds for keywd in sublist] # flatten list
    keywds = [keywd for keywd in keywds if keywd not in stpwds] # remove stopwords (rare but may happen due to n-gram breaking)
    keywds_stemmed = [stemmer.stem(keywd) for keywd in keywds]
    keywds_stemmed_unique = list(set(keywds_stemmed)) # remove duplicates (may happen due to n-gram breaking)
    keywds_gold_standard.append(keywds_stemmed_unique)
    
    if counter % round(len(keywd_names)/5) == 0:
        print(counter, 'files processed')

##############################
# precompute graphs-of-words #
##############################

### FILL THE GAP (use the terms_to_graph function, store the results in a list named 'gs') ###

##################################
# graph-based keyword extraction #
##################################

my_percentage = 0.33 # for PR and TF-IDF

method_names = ['kc','wkc','pr','tfidf']
keywords = dict(zip(method_names,[[],[],[],[]]))

for counter,g in enumerate(gs):
    # k-core
    core_numbers = core_dec(g,False)
    ### FILL THE GAPS (retain main core as keywords and append the resulting list to 'keywords['kc']') ###
    
    # weighted k-core
    ### FILL THE GAPS (repeat the procedure used for k-core) ###
    
    # PageRank
    pr_scores = zip(g.vs['name'],g.pagerank())
    pr_scores = sorted(pr_scores, key=operator.itemgetter(1), reverse=True) # in decreasing order
    numb_to_retain = int(len(pr_scores)*my_percentage) # retain top 'my_percentage' % words as keywords
    keywords['pr'].append([tuple[0] for tuple in pr_scores[:numb_to_retain]])
        
    if counter % round(len(gs)/5) == 0:
        print(counter)

#############################
# TF-IDF keyword extraction #
#############################

abstracts_cleaned_strings = [' '.join(elt) for elt in abstracts_cleaned] # to ensure same pre-processing as the other methods
tfidf_vectorizer = TfidfVectorizer(stop_words=stpwds)
### FILL THE GAP (call the .fit_transform() method and name the result 'doc_term_matrix') ###
terms = tfidf_vectorizer.get_feature_names()
vectors_list = doc_term_matrix.todense().tolist()

for counter,vector in enumerate(vectors_list):
    terms_weights = zip(terms,vector) # bow feature vector as list of tuples
    ### FILL THE GAP (keep only non zero values, i.e., the words in the document. store the results in a list named 'nonzero') ###
    nonzero = sorted(nonzero, key=operator.itemgetter(1), reverse=True) # in decreasing order
    numb_to_retain = int(len(nonzero)*my_percentage) # retain top 'my_percentage' % words as keywords
    keywords['tfidf'].append([tuple[0] for tuple in nonzero[:numb_to_retain]])
    
    if counter % round(len(vectors_list)/5) == 0:
        print(counter)

##########################
# performance comparison #
##########################

perf = dict(zip(method_names,[[],[],[],[]]))

for idx,truth in enumerate(keywds_gold_standard):
    for mn in method_names:
        ### FILL THE GAP (append to the 'perf[mn]' list by using the 'accuracy_metrics' function) ###

lkgs = len(keywds_gold_standard)

# print macro-averaged results (averaged at the collection level)
for k,v in perf.items():
    print(k + ' performance: \n')
    print('precision:', round(100*sum([tuple[0] for tuple in v])/lkgs,2))
    print('recall:', round(100*sum([tuple[1] for tuple in v])/lkgs,2))
    print('F-1 score:', round(100*sum([tuple[2] for tuple in v])/lkgs,2))
    print('\n')

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 3 (4 points): </b><br>
What can you say about the performance of the different approaches?
<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid red"> </hr>
Answer 3: </b><br>
Your answer here.
<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 4 (4 points): </b><br>
What are the respective (dis)advantages of (un)weighted k-core?
<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid red"> </hr>
Answer 4: </b><br>
Your answer here.
<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 5 (4 points): </b><br>
How could these approaches be improved? (check <a href='https://fragkiskos.me/papers/keywords_degeneracy_EMNLP_2016.pdf'>this article</a>)
<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid red"> </hr>
Answer 5: </b><br>
Your answer here.
<hr style="border:10px solid red"> </hr>
</font></h4>